In [1]:
# dependencies
import numpy as np
import pandas as pd

In [2]:
# support methods
# CODE GOES HERE

In [3]:
# __main__ routine
input_f = 'input/ages.csv'
output_f = 'output/clean_ages.parquet'

df = pd.read_csv(input_f, usecols=['record_id', 'age'])

To preview the data,

In [4]:
df.head()

,record_id,age
0,a72b20062e,69.0
1,dbc0f00485,101.0
2,cb4e5208b4,32.0
3,a4c36ded9d,None
4,812ed4562d,97.0


### assess missing data

Display names and data types of each column, along with the non-null value counts.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   record_id  564 non-null    object
 1   age        514 non-null    object
dtypes: object(2)
memory usage: 8.9+ KB


Okay, so roughly 50 rows have a value for `age` that pandas recognizes as null. Are there logical null values not included in this count?

In [6]:
df.age.unique()

array(['69.0', '101.0', '32.0', 'None', '97.0', '102.0', '48.0', '86.0',
       '12.0', '16.0', '81.0', '4.0', '84.0', nan, '17.0', '107.0',
       '75.0', '78.0', '83.0', '8.0', '25.0', '47.0', '5.0', '30.0',
       '11.0', '77.0', '37.0', '110.0', '89.0', '29.0', '34.0', '54.0',
       '21.0', '98.0', '19.0', '80.0', '15.0', '108.0', '100.0', '38.0',
       '57.0', '95.0', '67.0', '72.0', '43.0', '59.0', '46.0', '94.0',
       '85.0', '74.0', '20.0', '70.0', '104.0', '28.0', '109.0', '36.0',
       '27.0', '55.0', '62.0', '66.0', '14.0', '76.0', '56.0', '6.0',
       '26.0', '42.0', '79.0', '24.0', '105.0', '49.0', '99.0', '93.0',
       '9.0', '13.0', '39.0', '31.0', '33.0', '18.0', '65.0', '44.0',
       '58.0', '92.0', '51.0', '7.0', '23.0', '52.0', '50.0', '60.0',
       '96.0', '61.0', '103.0', '45.0', '22.0', '68.0', '41.0', '10.0',
       '64.0', '82.0', '90.0', '180.0', '63.0', '71.0', '40.0', '88.0',
       '106.0', '73.0', '91.0', '35.0', 'Unknown', '87.0', '53.0',
       '

Looks like `['None', 'Unknown', 'unknown']` are all currently treated as unique, non-missing values. Now that we have a list, we can write some code to identify these logical null values so they can be filtered or formatted.

In [7]:
def lazy_isnan(val):
    known = ['none', 'unknown']
    if str(val).lower() in known:
        return 1
    return 0


mask = [lazy_isnan(val) for val in df.age.values]
print(f'{sum(mask)} records are non-null, logical missing')
log_nan_df = df.loc[df.age.isin(['None', 'Unknown', 'unknown'])]
print('record_ids implicated:\n', log_nan_df.record_id.unique())

14 records are non-null, logical missing
record_ids implicated:
 ['a4c36ded9d' '2473f01571' '859371c786' '53e56691fe' '77c8184f67'
 'cfa1150f17' '2ef28d314a' 'f0483f255e' 'c8c06cfae2' '093f0b067a'
 'b87bed1295' '3219b5be78' '9741f79aca' '916f5b10fe']


### assess variable range

In [8]:
df.age.describe()

count      514
unique     111
top       66.0
freq        10
Name: age, dtype: object

From `.describe()`, we get some general info about the characteristics of this age data. Notably, our `max` value is 180... Does that make sense? Let's clean up this data and then see how many values are unusually high like that.

### apply formatting, cleaning methods

In [9]:
def format_nan(val):
    if pd.isnull(val):
        return None
    elif lazy_isnan(val):
        return None
    else:
        if '.0' in val:
            form = val.replace('.0', '')
            return int(form)


def format_age_col(df):
    copy = df.copy()
    copy['age'] = df.age.apply(format_nan)
    return copy


clean_df = format_age_col(df)

In [10]:
clean_df.loc[clean_df.age >= 120.0]

,record_id,age
302,ec7f1f6506,180.0


Only one! Okay, since the descriptive statistics are otherwise within reason for what we'd expect from a collection of human ages, we should proceed with cleaning the feature for now. Let's make a note of that record_id and follow up on it later, it might be a data entry error and we can look at the source material to verify.

### generate categorical variable

In [11]:
def get_age_group(val):
    if pd.notnull(val):
        if val >= 0:
            if val < 18:
                return 'Under 18'
            elif val < 60:
                return 'Adult'
            else:
                return 'Senior'
    return None


def age_group_asserts():
    test_cases = [
        (None, None), 
        (np.nan, None), 
        (120.0, 'Senior'), 
        (-1, None), 
        (12, 'Under 18'), 
        (18, 'Adult'), 
        (45, 'Adult'), 
        (60, 'Senior')
    ]
    for i in range(len(test_cases)):
        test_val = test_cases[i][0]
        expected = test_cases[i][1]
        assert get_age_group(test_val) == expected
    return 1


def make_age_group_col(df):
    assert 'age' in df.columns
    assert age_group_asserts()
    copy = df.copy()
    copy['age_group'] = copy.age.apply(get_age_group)
    return copy


group_df = make_age_group_col(clean_df)

Great! Now that we have a useful category to aid in later analysis, let's get a quick snapshot of the distribution.

In [12]:
group_df.age_group.value_counts()

Senior      250
Adult       188
Under 18     62
Name: age_group, dtype: int64

Excellent! Now, if we wanted to make sure this data does not change, or prompt an update to this script if new data is added, then we could use the frequency info in an assert statement. Otherwise, it's useful to log this info or capture it in a magic-numbers doc with each iteration of the script.

## A word about Jupyter notebooks
For principled data processing, the best way to format a jupyter notebook is to iteratively develop the program and re-format as you go. When a method is working as expected, copy it into the "#support methods" cell and add the corresponding method call to the "#main" cell. That way, the first three cells of your notebook should follow the ideal python script style: 
1. load dependencies
2. load custom methods
3. load instructions for main.

Outside of active development, I try to limit the number of cells left with output displayed, instead trimming these while re-formatting. In the case of consciously formatting a notebook like a report, standing output can be a handy way to support the flow of info, but it can become overwhelming and confusing if you aren't careful. Prune wisely!